In [14]:
import pandas as pd
import re
import os


def convert_to_csv(INPUT_PATH = 'KolejnyTest/5M/x/report_5_x.csr',OUTPUT_PATH = 'KolejnyTest/5M/x/report_5_x.csv'):
    # === CONFIG ===
    # INPUT_PATH = 'KolejnyTest/5M/x/report_5_x.csr'   # Input file path
    # OUTPUT_PATH = 'KolejnyTest/5M/x/report_5_x.csv'  # Output file path

    # === READ FILE ===
    with open(INPUT_PATH, 'r',encoding='windows-1250') as f:
        lines = [line.strip() for line in f if line.strip()]

    divider_indexes = []
    for i,line in enumerate(lines):
        if lines[i][0] == '#':
            divider_indexes.append(i)
    divider_indexes.append(len(lines))


    for i in range(10):
        lines_iter = lines[divider_indexes[i]:divider_indexes[i+1]]

        out_path =  f"{OUTPUT_PATH[:-4]}_{i+1}.csv"
        metadata_line_index = next(i for i, line in enumerate(lines_iter) if 'obiekt' in line.lower())
        metadata_values = lines_iter[metadata_line_index + 1].split()

        is_empty = False
        try:
            obiektow_w, udzial_punktow, pole_obrazu = metadata_values[:3]
        except Exception as e:
            obiektow_w, pole_obrazu = metadata_values[:2]
            udzial_punktow = 0.00
            is_empty = True

        if not is_empty:
            # === FIND HEADER INDEX ===
            header_line_index = next(i for i, line in enumerate(lines_iter) if 'nr.' in line.lower())
            header_line = lines_iter[header_line_index]
            column_names = re.split(r'\t+', header_line.strip())

            # Standardize column names (remove units and special characters)
            column_names = [
                re.sub(r'[\(\):°�˛]', '', col).strip().lower().replace(' ', '_')
                for col in column_names
            ]

            # === PARSE DATA ===
            data_lines = lines_iter[header_line_index + 1:]
            data = [re.split(r'\t+', line.strip()) for line in data_lines]

            # === CREATE DATAFRAME ===
            df = pd.DataFrame(data, columns=column_names)
        else:
            df = pd.DataFrame()

        # === ADD METADATA COLUMNS ===
        df['liczenie_obiektow_w'] = obiektow_w
        df['udzial_punktow'] = udzial_punktow
        df['pole_obrazu_mm2'] = pole_obrazu

        # === CONVERT TYPES ===
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='ignore')

        # === SAVE TO UTF-8 CSV ===
        df.to_csv(out_path, index=False, encoding='utf-8')

        print(f'✔ Data converted and saved to: {out_path}')

In [4]:
import pandas as pd
import cv2
import numpy as np

def process_one_image(CSV_PATH = 'KolejnyTest/5M/x/report_5_x.csv',IMAGE_PATH = f'KolejnyTest/5M/x/a.jpg'   ,OUTPUT_PATH = f'KolejnyTest/5M/x/a_marked.jpg'):
    # === CONFIG ===
            # Path to the original image
        # Where to save the output image

    # === LOAD IMAGE AND DATA ===
    img = cv2.imread(IMAGE_PATH)
    df = pd.read_csv(CSV_PATH)

    height, width = img.shape[:2]

    # === DRAW OBJECTS ===
    for i, row in df.iterrows():
        try:
            # Extract pixel coordinates
            x = int(row['xpix.'])
            y = int(row['ypix.'])
            
            # Extract size in mm → convert to pixels if needed (assuming 1mm = 10px for now, adjust as needed)
            width_mm = float(row['szerokośćmm'])
            height_mm = float(row['długośćmm'])
            angle = -float(row['kąt'])+90  # OpenCV uses negative angle for rotation

            mm_to_px = 10  # ⬅️ Adjust this scaling factor based on your image resolution

            width_px = int(width_mm * mm_to_px)
            height_px = int(height_mm * mm_to_px)

            # Draw ellipse representing the object
            center = (x, height-y)
            # axes = (1, 1)
            axes = (width_px // 2, height_px // 2)
            color = (0, 255, 0)  # Green
            thickness = 2

            cv2.ellipse(img, center, axes, angle, 0, 360, color, thickness)

            # Optional: draw ID
            cv2.putText(img, str(int(row['nr.'])), (x + 5, height - y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
        except Exception as e:
            print(e)

    # === SAVE RESULT ===
    cv2.imwrite(OUTPUT_PATH, img)
    print(f"Image saved to {OUTPUT_PATH}")

In [17]:
def convert_full_folder(directory_path="KolejnyTest/5M",test_sign='x',num=1):
    convert_to_csv(INPUT_PATH = f'{directory_path}/raport_{num}_{test_sign}.txt',OUTPUT_PATH = f'{directory_path}/{test_sign}/raport_{num}_{test_sign}.csv')
    for i in range(1,11):
        directory_images = f'{directory_path}/{test_sign}/a{i}'
        if not os.path.exists(directory_images):
            os.makedirs(directory_images)
        for j in range(1,11):
            CSV_PATH = f'{directory_path}/{test_sign}/raport_{num}_{test_sign}_{j}.csv'
            OUTPUT_PATH = f'{directory_images}/a{i}_{j}_marked.jpg'
            process_one_image(i,CSV_PATH=CSV_PATH,OUTPUT_PATH=OUTPUT_PATH,IMAGE_PATH = f'{directory_path}/{test_sign}/a{i}.jpg')

In [22]:
for i in [1,2,3,16,17,18,19,20,21,22]:
    convert_full_folder(directory_path=f"Kaffioyra_2024_dane/Agatka/{i}",test_sign='y',num=i)

/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='i

✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_1.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_2.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_3.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_4.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_5.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_6.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_7.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_8.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_9.csv
✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/1/y/raport_1_y_10.csv
Image saved to Kaffioyra_2024_dane/Agatka/1/y/a1/a1_1_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/1/y/a1/a1_2_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/1/y/a1/a1_3_marked.jpg
could not conv

/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='i

Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_3_marked.jpg
cannot convert float NaN to integer
cannot convert float NaN to integer
invalid literal for int() with base 10: 'udział punktów'
invalid literal for int() with base 10: '100.00'
invalid literal for int() with base 10: 'x(pix.):'
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_4_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_5_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_6_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_7_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_8_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_9_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a1/a1_10_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a2/a2_1_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a2/a2_2_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/2/y/a2/a2_3_marked.jpg
cannot convert float NaN to integer
cannot convert floa

/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='i

...

✔ Data converted and saved to: Kaffioyra_2024_dane/Agatka/20/y/raport_20_y_10.csv
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_1_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_2_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_3_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_4_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_5_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_6_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_7_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_8_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_9_marked.jpg
could not convert string to float: '******'
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/20/y/a1/a1_10_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Ag

/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='i

could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a1/a1_7_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a1/a1_8_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a1/a1_9_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a1/a1_10_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_1_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_2_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_3_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_4_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_5_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_6_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/21/y/a2/a2_7_marked.jpg
Image sav

/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/tmp/ipykernel_33357/1053189745.py:65: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='i

Image saved to Kaffioyra_2024_dane/Agatka/22/y/a1/a1_8_marked.jpg
could not convert string to float: '******'
could not convert string to float: '******'
could not convert string to float: '******'
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a1/a1_9_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a1/a1_10_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_1_marked.jpg
could not convert string to float: '******'
could not convert string to float: '******'
could not convert string to float: '******'
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_2_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_3_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_4_marked.jpg
could not convert string to float: '******'
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_5_marked.jpg
Image saved to Kaffioyra_2024_dane/Agatka/22/y/a2/a2_6_marked.jpg
Image sav

In [5]:
process_one_image(CSV_PATH="raport_5_x_2.csv",IMAGE_PATH="dataset0/images/train/a1.jpg",OUTPUT_PATH="test.jpg")

Image saved to test.jpg
